<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

**Библиотеки**

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.model_selection import TimeSeriesSplit

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

**Загружаем датасет**

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

**Посмотрим данные**

In [3]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропусков нет

In [5]:
display(data['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

8.841344371679229

Классы в датасете несбалансированы. Необходимо это учитывать при дальнейшей обучении моделей.

**Лемматизация**

*Сначала хотел через BERT, а потом через Mystem, но ядро умирает, рабочий ПК не вывозит*

создадим корпус

In [7]:
corpus = list(data['text'])

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def clear_text(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return re_list

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

m = WordNetLemmatizer()

def lemmatize_text(text):
    word_list = nltk.word_tokenize(text)
    
    return ' '.join([m.lemmatize(w, get_wordnet_pos(w)) for w in word_list])

In [10]:
print("Проверим")
print()
print("Исходный текст:")
print(corpus[0])
print()
print("Очищенный и лемматизированный текст:")
print(lemmatize_text(clear_text(corpus[0])))

Проверим

Исходный текст:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Очищенный и лемматизированный текст:
Explanation Why the edits make under my username Hardcore Metallica Fan be revert They be n't vandalism just closure on some GAs after I vote at New York Dolls FAC And please do n't remove the template from the talk page since I 'm retire now


In [11]:
data['text'] = data['text'].apply(clear_text)

In [12]:
data['lemm_text'] = data['text'].apply(lemmatize_text)

In [13]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation Why the edits made under my userna...,0,Explanation Why the edits make under my userna...
1,1,D'aww He matches this background colour I'm se...,0,D'aww He match this background colour I 'm see...
2,2,Hey man I'm really not trying to edit war It's...,0,Hey man I 'm really not try to edit war It 's ...
3,3,More I can't make any real suggestions on impr...,0,More I ca n't make any real suggestion on impr...
4,4,You sir are my hero Any chance you remember wh...,0,You sir be my hero Any chance you remember wha...


**Разделим данные на выборки**

In [14]:
features = corpus
target = data['toxic'].values
RANDOM_STATE = 12345
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=RANDOM_STATE)

Создадим счётчик, указав в нём стоп-слова:

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
features_test = count_tf_idf.transform(features_test)

## Обучение

Классы в датасете несбалансированы. Необходимо это учитывать при обучении моделей

**Построем модели со значением метрики качества F1 не меньше 0.75**

**Модель "Логистическая регрессия"**

In [17]:
%%time
lr_model = LogisticRegression()
hyperparams = [{'C':[0.1, 1, 10],
                'class_weight':['balanced']}]
lr_clf = GridSearchCV(lr_model, hyperparams, scoring='f1',cv=3)
lr_clf.fit(features_train, target_train)
lr_score = lr_clf.best_score_
print("Лучшие параметры модели:")
print()
LR_best_params = lr_clf.best_params_
print(LR_best_params)
print()
print('F1 модели Логистическая регрессия:', lr_score)
print()

Лучшие параметры модели:

{'C': 10, 'class_weight': 'balanced'}

F1 модели Логистическая регрессия: 0.7626180553601154

CPU times: user 2min 4s, sys: 4min 45s, total: 6min 49s
Wall time: 6min 50s


**Модель "Дерево решений"**

In [18]:
%%time
dtc_model = DecisionTreeClassifier()
hyperparams = [{'max_depth':[x for x in range(1,20)],
                'random_state':[RANDOM_STATE],
                'class_weight':['balanced']}]
dtc_clf = GridSearchCV(dtc_model, hyperparams, scoring='f1',cv=3)
dtc_clf.fit(features_train, target_train)
print("Лучшие параметры модели:")
print()
dtc_best_params = dtc_clf.best_params_
print(dtc_best_params)
print()
dtc_score = dtc_clf.best_score_
print('F1 модели Дерево решений:', dtc_score)
print()

Лучшие параметры модели:

{'class_weight': 'balanced', 'max_depth': 19, 'random_state': 12345}

F1 модели Дерево решений: 0.5912005464587206

CPU times: user 11min 18s, sys: 2.85 s, total: 11min 21s
Wall time: 11min 22s


**Модель "CatBoost"**

In [19]:
cat_model = CatBoostClassifier(verbose=False, iterations=250)
cat_model.fit(features_train, target_train)
cat_score = cross_val_score(cat_model,
                                         features_train, 
                                         target_train, 
                                         cv=3, 
                                         scoring='f1').mean()
print('F1 модели CatBoost', cat_score)

F1 модели CatBoost 0.729487782219269


In [23]:
column=['Модели по метрике качества F1']

comparison_tabl = pd.DataFrame(index=['Модели по метрике качества F1'], columns=['Логистическая регрессия','Дерево решений','CatBoost'])
comparison_tabl['Логистическая регрессия'] = lr_score
comparison_tabl['Дерево решений'] = dtc_score
comparison_tabl['CatBoost'] = cat_score
comparison_tabl

,Логистическая регрессия,Дерево решений,CatBoost
Модели по метрике качества F1,0.762618,0.591201,0.729488


**Лучший результат на тренировочной выборке показала модель "Логистическая регрессия"**

**Теперь обучим лучшую модель на тестовой выборке**

In [21]:
%%time
lr_test_model = LogisticRegression()
lr_test_model.set_params(**LR_best_params)
lr_test_model.fit(features_train, target_train)
prediction = lr_test_model.predict(features_test)
f1 = f1_score(target_test, prediction)
print('F1 модели "Логистическая регрессия" на тестовой выборке:', f1)
print()

F1 модели "Логистическая регрессия" на тестовой выборке: 0.7756048095031146

CPU times: user 14.8 s, sys: 36.1 s, total: 50.9 s
Wall time: 50.9 s


**F1 модели соответствует условиям поставленной задачи**

## Выводы

- Проведена предподготовка данных
- Тект был очищен и лематизирован
- Обучены разные модели и лучшая из них обучена на тестовой выборке
- F1 на тестовой виборке было больше 0,75
- Для поставленной задичи лучше подходит модель LogisticRegression

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны